In [1]:
import warnings
warnings.filterwarnings("ignore")
from pyspark.streaming import StreamingContext

ssc = StreamingContext(sc, 5)
# Provide a checkpointing directory. Required for stateful transformations
ssc.checkpoint("checkpoint")

numPartitions = 8
rdd = sc.textFile('adj_noun_pairs.txt', numPartitions)
rddQueue = rdd.randomSplit([1]*10, 123)
lines = ssc.queueStream(rddQueue)

# FILL IN YOUR CODE
def updateFunc(newValues, runningCount):
    if runningCount is None: 
        return newValues
    else:
        if len(newValues) > len(runningCount):
            return newValues
        else:
            return runningCount
    # add the new values with the previous running count to get the new count
    
running_counts = lines.map(lambda line: (line.split(" ")[0],line.split(" ")[1]))\
                      .filter(lambda l: len(l) > 1)\
                      .reduceByKey(lambda x,y : x if len(x) >= len(y) else y)\
                      .updateStateByKey(updateFunc)

#just to sort according to the length
counts_sorted = running_counts.transform(lambda rdd: rdd.sortBy(lambda x: len(x[1]), False))

def printResults(rdd):
    print(rdd.take(5))
    print('good:', rdd.lookup('good')[0][0])

counts_sorted.foreachRDD(printResults)


ssc.start()
ssc.awaitTermination(50)
ssc.stop(False)
print("Finished")

21/11/09 15:25:16 WARN QueueInputDStream: queueStream doesn't support checkpointing


[('social', ['constructionism']), ('desirable', ['characteristic']), ('other', ['socialist-revolutionary']), ('nonviolent', ['resistance']), ('earthly', ['consideration'])]
good: morning/afternoon


21/11/09 15:25:31 WARN QueueInputDStream: queueStream doesn't support checkpointing


[('social', ['constructionism']), ('desirable', ['characteristic']), ('other', ['socialist-revolutionary']), ('nonviolent', ['resistance']), ('earthly', ['consideration'])]


21/11/09 15:25:33 WARN QueueInputDStream: queueStream doesn't support checkpointing
21/11/09 15:25:33 WARN QueueInputDStream: queueStream doesn't support checkpointing
21/11/09 15:25:33 WARN QueueInputDStream: queueStream doesn't support checkpointing


...

[('social', ['constructionism']), ('desirable', ['characteristic']), ('other', ['socialist-revolutionary']), ('nonviolent', ['resistance']), ('earthly', ['consideration'])]
good: morning/afternoon


21/11/09 15:25:57 WARN QueueInputDStream: queueStream doesn't support checkpointing
21/11/09 15:26:01 WARN QueueInputDStream: queueStream doesn't support checkpointing


[('social', ['constructionism']), ('desirable', ['characteristic']), ('other', ['socialist-revolutionary']), ('nonviolent', ['resistance']), ('earthly', ['consideration'])]
good: morning/afternoon


21/11/09 15:26:02 WARN QueueInputDStream: queueStream doesn't support checkpointing
21/11/09 15:26:06 WARN BatchedWriteAheadLog: BatchedWriteAheadLog Writer queue interrupted.
21/11/09 15:26:06 WARN QueueInputDStream: queueStream doesn't support checkpointing


[('social', ['constructionism']), ('desirable', ['characteristic']), ('other', ['socialist-revolutionary']), ('nonviolent', ['resistance']), ('earthly', ['consideration'])]
good: morning/afternoon
Finished


In [ ]:
numPartitions = 8
rdd = sc.textFile('adj_noun_pairs.txt', numPartitions)
rdd.map(lambda line: (line.split(" ")[0],line.split(" ")[1])).filter(lambda l: len(l) > 1).reduceByKey(lambda x,y : x if len(x) >= len(y) else y).take(5)

In [3]:
from pyspark.streaming import StreamingContext

# 计算累积的word count

# Stateful word count

ssc = StreamingContext(sc, 5)
# Provide a checkpointing directory.  Required for stateful transformations
ssc.checkpoint("checkpoint")

rdd = sc.textFile('adj_noun_pairs.txt', 8)
rddQueue = rdd.randomSplit([1]*10, 123)
lines = ssc.queueStream(rddQueue)

def updateFunc(newValues, runningCount):
    #new Values: 新的stream中该单词的count
    #runningCount : 旧有的统计中的单词的count
    if runningCount is None: #原来的统计中该单词不存在
        runningCount = 0
    return sum(newValues, runningCount)
    # add the new values with the previous running count to get the new count

running_counts = lines.flatMap(lambda line: line.split(" "))\
                      .map(lambda word: (word, 1))\
                      .updateStateByKey(updateFunc)

counts_sorted = running_counts.transform(lambda rdd: rdd.sortBy(lambda x: x[1], False))

def printResults(rdd):
    print("Total distinct words: ", rdd.count())
    print(rdd.take(5))
    print('refinery:', rdd.lookup('refinery')[0])

counts_sorted.foreachRDD(printResults)

ssc.start()
ssc.awaitTermination(50)
ssc.stop(False)
print("Finished")

21/12/07 16:08:31 WARN TaskSetManager: Lost task 0.0 in stage 355.0 (TID 514, 192.168.68.102, executor driver): TaskKilled (Stage cancelled)
21/12/07 16:08:31 WARN TaskSetManager: Lost task 1.0 in stage 355.0 (TID 515, 192.168.68.102, executor driver): TaskKilled (Stage cancelled)
21/12/07 16:08:31 WARN TaskSetManager: Lost task 6.0 in stage 355.0 (TID 520, 192.168.68.102, executor driver): TaskKilled (Stage cancelled)
21/12/07 16:08:31 WARN TaskSetManager: Lost task 7.0 in stage 354.0 (TID 513, 192.168.68.102, executor driver): TaskKilled (Stage cancelled)
21/12/07 16:08:31 ERROR JobScheduler: Error generating jobs for time 1638864510000 ms
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/shawin/hadoop/spark-3.0.3/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/Users/shawin/hadoop/spark-3.0.3/python/pyspark/streaming/dstream.py", line 297, in <lambda>
    func = lambda t, rdd:

Py4JJavaError: An error occurred while calling o2788.start.
: java.lang.IllegalStateException: Only one StreamingContext may be started in this JVM. Currently running StreamingContext was started atorg.apache.spark.streaming.api.java.JavaStreamingContext.start(JavaStreamingContext.scala:557)
sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
java.lang.reflect.Method.invoke(Method.java:498)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:282)
py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
py4j.commands.CallCommand.execute(CallCommand.java:79)
py4j.GatewayConnection.run(GatewayConnection.java:238)
java.lang.Thread.run(Thread.java:748)
	at org.apache.spark.streaming.StreamingContext$.or